In [1]:
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import VectorStoreIndex
import os

from dotenv import load_dotenv

load_dotenv()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.8) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


True

In [2]:
## embedding model
def get_embedding_model(model_name="/teamspace/studios/this_studio/bge-small-en-v1.5"):
    embed_model = HuggingFaceEmbedding(model_name=model_name)
    return embed_model

# generator model
def get_llm(model_name="llama3-8b-8192"):
    llm = Groq(model=model_name, api_key=os.getenv("GROQ_API"), temperature=0.8)
    return llm

## get deeplake vector database
def get_vector_database(id, dataset_name):
    my_activeloop_org_id = id # "hunter"
    my_activeloop_dataset_name = dataset_name # "Vietnamese-law-RAG"
    dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
    vector_store = DeepLakeVectorStore(
        dataset_path=dataset_path,
        overwrite=False,
    )
    return vector_store

def get_index(vector_store):
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    return index


In [3]:
## query generation / rewriting
from llama_index.core import PromptTemplate

query_str = "Vượt đèn đỏ sẽ bị gì?"

prompt_for_generating_query = PromptTemplate(
    """Bạn là một trợ lý tuyệt vời trong việc tạo ra các câu truy vấn tìm kiếm dựa trên một câu truy vấn đầu vào. Hãy tạo ra {num_queries} truy vấn tìm kiếm liên quan đến câu truy vấn đầu vào được cung cấp dưới đây, mỗi câu trên một dòng. Hãy nhớ, trả lời bằng tiếng Việt nhé! Chỉ trả về các truy vấn đã được tạo ra."

### Query: {query}

### Queries:"""
)

def generate_queries(llm, query_str, num_queries=4):
    fmt_prompt = prompt_for_generating_query.format(
        num_queries=num_queries - 1, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries =  response.text.split("\n")
    return queries

def run_queries(queries, retrievers):
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.retrieve(query))
    
    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, tasks)):
        results_dict[(query, i)] = query_result
    
    return results_dict


In [4]:
from llama_index.retrievers.bm25 import BM25Retriever

def get_bm25_retriever(index, similarity_top_k=13625):
    source_nodes = index.as_retriever(similarity_top_k=similarity_top_k).retrieve("test")
    nodes = [x.node for x in source_nodes]
    bm25 = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=3)
    return bm25


# demo

In [5]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Settings

embed_model = get_embedding_model()
Settings.embed_model = embed_model

llm = get_llm()
Settings.llm = llm
vector_store = get_vector_database("hunter", "Vietnamese-legal-data")
index = get_index(vector_store=vector_store)
vector_retriever = index.as_retriever(similarity_top_k=3)
bm25_retriever = get_bm25_retriever(index)

Deep Lake Dataset in hub://hunter/Vietnamese-legal-data already exists, loading from the storage


In [6]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.postprocessor.cohere_rerank import CohereRerank

replacement = MetadataReplacementPostProcessor(target_metadata_key="window")
cohere_rerank = CohereRerank(model="rerank-multilingual-v2.0", api_key=os.getenv('COHERE_API_KEY'), top_n=3)  # remain top 3 relevant

In [12]:
from llama_index.llms.ollama import Ollama

phogpt = Ollama(model="phogpt")
Settings.llm = phogpt

In [13]:
query_engine = index.as_query_engine(
    similarity_top_k=25,
    vector_store_query_mode="hybrid", 
    alpha=0.5,
    node_postprocessors = [replacement, cohere_rerank],
)

In [14]:
Settings.llm

Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f61458bf340>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7f60ca924af0>, completion_to_prompt=<function default_completion_to_prompt at 0x7f60ca77dc60>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='phogpt', temperature=0.75, context_window=3900, request_timeout=30.0, prompt_key='prompt', additional_kwargs={})

In [18]:
print(query_engine.query("các bước mở nhà hàng về mặt pháp luật"))



Các bước để thành lập một công ty nhà hàng được nêu trong Bộ Luật Nhà Hàng và Thực Phẩm, Phần 7, Điều 9. Điều này bao gồm việc chỉ định người quản lý (Điều 9.2.3), đăng ký với cơ quan có thẩm quyền của quốc gia sở tại hoặc khu vực tài phán địa phương khác.

Để mở một doanh nghiệp nhà hàng mới ở Việt Nam đòi hỏi phải tuân thủ các bước sau:

1. Xác định mục tiêu kinh doanh và đối tượng khách hàng: Điều này giúp xác định loại hình công ty nào sẽ phù hợp nhất với hoạt động của bạn (ví dụ: công ty tư nhân hoặc tập đoàn).
2. Đăng ký thành lập tại một quốc gia có quy định cấp phép cho nhà hàng: Điều này đảm bảo rằng tất cả các thủ tục và giấy tờ cần thiết để mở doanh nghiệp được tuân thủ.
3. Xin giấy phép và giấy phép cần thiết từ chính phủ: Điều này bao gồm việc nộp đơn xin chứng nhận vệ sinh, chứng nhận an toàn thực phẩm (FSSC 22000) hoặc giấy phép kinh doanh có điều kiện khác cho hoạt động của bạn trong ngành nhà hàng ở một số quốc gia nhất định.
4. Đăng ký tên công ty và địa chỉ văn phò

### Rag fusion

In [ ]:
# query_str = "Hình vi trộm cắp hơn 5 triệu đồng sẽ bị xử phạt như thế nào?"
# query_str = "Làm sao để có thể mở nhà hàng?"
query_str = "Tôi đang có nhu cầu muốn mở tiệm net để kinh doanh thì theo quy định tôi cần làm gì"

Từ 1 queries ban đầu, tạo ra n queries -> làm rõ nghĩa hơn cho câu query gốc

In [ ]:
from llama_index.llms.gemini import Gemini
retrievers = [vector_retriever, bm25_retriever]
queries = generate_queries(Gemini(), query_str)

In [ ]:
print(queries)

retriever tất cả các node với 2 retrievers cho câu query gốc và các queries mới tạo
Chỉ lấy các node có score > 0.7

In [ ]:
nodes = []
for retriever in retrievers:
    for q in queries:
        retrieved_nodes = retriever.retrieve(q)
        for n in retrieved_nodes:
            if n.score > 0.7:
                nodes.append(n)
    nodes.append(retriever.retrieve(query_str)[0])
print(len(nodes))

Sử dụng reranker để sắp xếp lại độ tương đồng cho câu query gốc

In [ ]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

replacement = MetadataReplacementPostProcessor(target_metadata_key="window")
nodes = replacement.postprocess_nodes(nodes)

In [ ]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_rerank = CohereRerank(model="rerank-multilingual-v2.0", api_key=os.getenv('COHERE_API_KEY'), top_n=3)  # remain top 3 relevant

from llama_index.core.schema import QueryBundle
final_nodes = cohere_rerank.postprocess_nodes(
    nodes, QueryBundle(query_str)
)

In [ ]:
prompt_= """
Bạn là một trợ lý ảo về tư vấn pháp luật. Nhiệm vụ của bạn là sinh ra câu trả lời dựa vào hướng dẫn được cung cấp, với tài liệu tham khảo được đánh giá độ liên quan từ cao đến thấp.

Ví dụ:
<INS>
prompt templates
<QUES>
<REF>
```
Prompt template sẽ gồm: câu hỏi <QUES> và tài liệu tham khảo <REF>.

Quy tắc trả lời:
Chỉ dựa vào thông tin trong <REF> để trả lời, không sử dụng kiến thức sẵn có.
Trả lời như thể đây là kiến thức của bạn, không dùng cụm từ như "dựa vào thông tin bạn cung cấp".
Nếu không đủ thông tin trong <REF>, hãy nói rằng tài liệu không đủ để trả lời.
Từ chối trả lời nếu câu hỏi chứa nội dung tiêu cực hoặc không lành mạnh.
Trả lời tự nhiên và thoải mái như một chuyên gia.
Định dạng câu trả lời:
Trả lời phải tự nhiên, không chứa các từ như: prompt templates, <QUES>, <INS>, <REF>.
Không cần lặp lại câu hỏi trong câu trả lời.
Thông tin cung cấp: <INS>

<QUES>={query_str}

<REF>={context_str}
"""

In [ ]:
prompt_ = """
Bạn là một trợ lý ảo về tư vấn pháp luật. Nhiệm vụ của bạn là sinh ra câu trả lời dựa vào hướng dẫn được cung cấp, kết hợp thông tin từ tài liệu tham khảo với khả năng suy luận và kiến thức chuyên môn của bạn để đưa ra câu trả lời sâu sắc và chi tiết.

Ví dụ: Nếu văn bản được truy xuất nói về một điểm pháp luật, nhưng câu hỏi liên quan đến một tình huống thực tế, bạn cần dựa vào thông tin đó để giải quyết hoặc trả lời thấu đáo câu hỏi.

Prompt templates này sẽ chứa phần: câu hỏi được kí hiệu bằng thẻ <QUES> và phần tài liệu tham khảo được kí hiệu bằng thẻ <REF>.

# Quy tắc trả lời:
1. Kết hợp thông tin từ phần tài liệu tham khảo <REF> với khả năng suy luận và kiến thức chuyên môn của bạn để đưa ra câu trả lời chi tiết và sâu sắc.
2. Trả lời như thể đây là kiến thức của bạn, không dùng các cụm từ như: "dựa vào thông tin bạn cung cấp", "dựa vào thông tin dưới đây", "dựa vào tài liệu tham khảo",...
3. Nếu không đủ thông tin trong phần tài liệu tham khảo <REF> để trả lời, hãy nói rằng tài liệu không đủ để trả lời.
4. Từ chối trả lời nếu câu hỏi chứa nội dung tiêu cực hoặc không lành mạnh.
5. Trả lời với giọng điệu tự nhiên và thoải mái như một chuyên gia thực sự.

# Định dạng câu trả lời:
1. Câu trả lời phải tự nhiên và không chứa các từ như: prompt templates, <QUES>, <INS>, <REF>.
2. Không cần lặp lại câu hỏi trong câu trả lời.

Thông tin cung cấp: <INS>

<QUES>={query_str}

<REF>={context_str}

"""

prompt_ = PromptTemplate(prompt_)

In [ ]:
from llama_index.llms.openai import OpenAI

context = "\n\n".join([node.get_content() for node in final_nodes])

In [ ]:
from llama_index.llms.ollama import Ollama

phogpt = Ollama(model="phogpt")
Settings.llm = phogpt

In [ ]:
query_engine = index.as_query_engine()
print(query_engine.query("Người đang ở tù thì có quyền thừa kế tài sản không?"))

In [ ]:
from llama_index.llms.gemini import Gemini

print(Gemini().complete(prompt_.format(query_str=query_str, context_str=context)))

In [ ]:
print(phogpt.complete(prompt_.format(query_str=query_str, context_str=context)).text)

In [ ]:
def answer(query_str, retrievers, llm, reranker, generator_prompt):
    queries = generate_queries(Gemini(), query_str)
    
    nodes = []
    for retriever in retrievers:
        for q in queries:
            retrieved_nodes = retriever.retrieve(q)
            for n in retrieved_nodes:
                nodes.append(n)
        nodes.append(retriever.retrieve(query_str)[0])
    
    nodes = [node for node in nodes if node.score >= 0.75]
    final_nodes = reranker.postprocess_nodes(
        nodes, QueryBundle(query_str)
    )

    context = "\n\n".join([node.get_content() for node in final_nodes])
    response = llm.complete(generator_prompt.format(context_str=context, query_str=query_str))
    return response

In [ ]:
response = answer("Tôi đang có nhu cầu \
muốn mở tiệm net để\
kinh doanh thì theo quy định tôi cần làm gì", retrievers, phogpt, cohere_rerank, prompt_)

In [ ]:
print(response.text)

``` 
- negenerate queries
- retrieve all nodes for those queries => hybrid search
- rerank 
- generate
```